In [2]:
import pandas as pd
import yfinance as yf
from datetime import date, datetime, timedelta
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
start_date = '2021-1-1'
end_date = '2024-1-1'
print("Start Date:",start_date," End Date:",end_date)

In [3]:
# import yfinance as yf
# import pandas as pd

# # Download stock data for ZOMATO.NS
# df = yf.download('ZOMATO.NS', start='2021-01-01', end='2024-12-17')

# # Reset the index to create a 'Date' column
# df.reset_index(inplace=True)

# # Display the first few rows of the DataFrame
# print(df.head())

In [ ]:
ticker = 'ZOMATO.NS'

df = yf.download(ticker, start=start_date, end=end_date)
df

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df['Open'].plot()

In [ ]:
df['Close'].plot()

In [ ]:
df['Close Rolling 10'] = df['Close'].rolling(window=10,min_periods=1).mean()
df.head(14)

In [ ]:
df[['Close','Close Rolling 10']].plot()

In [12]:
df['Close Rolling 30'] = df['Close'].rolling(window=30,min_periods=1).mean()

In [ ]:
df['Close Rolling 50'] = df['Close'].rolling(window=50,min_periods=1).mean()
df

In [ ]:
df[['Close','Close Rolling 10','Close Rolling 30','Close Rolling 50']].plot()

In [ ]:
#cummulative MOving average
df['CMA']=df['Close'].expanding().mean()
df

In [ ]:
df[['Close','CMA']].plot()

In [17]:
def adf_test(series):
    result=adfuller(series)
    print('ADF Statistics: {}'.format(result[0]))
    print('p- value: {}'.format(result[1]))
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [ ]:
adf_test(df['Close'])

In [ ]:
df['First Difference'] = df['Close']-df['Close'].shift(1)
df.head()

In [ ]:
adf_test(df['First Difference'].dropna())

In [ ]:
df['First Difference'].dropna().plot()

In [ ]:
acf = plot_acf(df["First Difference"].dropna())
pacf = plot_pacf(df["First Difference"].dropna())

In [ ]:
step = auto_arima(df['First Difference'].dropna(),trace=True,suppress_warnings=True)
step.summary()

In [ ]:
model_ARIMA=ARIMA(df['Close'],order=(15,2,3))
model_ARIMA_fit = model_ARIMA.fit()

In [ ]:
model_ARIMA_fit.summary()

In [ ]:
df1 = yf.download(ticker,start='2024-12-1',end=date.today())
df1

In [27]:
pred_arima=model_ARIMA_fit.predict(start=datetime(2023,12,8),end=datetime(2023,12,29))
residuals=df['Close']-pred_arima

#df.head()

#df.tail()

In [ ]:
df.tail(15)

In [ ]:
model_ARIMA_fit.resid.plot()

In [ ]:
model_ARIMA_fit.resid.plot(kind='kde')

In [31]:
df['Predicted_ARIMA']=pred_arima

In [ ]:
df.tail(15)

In [ ]:
df['Predicted_ARIMA']=pred_arima
print(df['Predicted_ARIMA'].dropna())

In [ ]:
df[['Close','Predicted_ARIMA']].plot()

In [ ]:
forecast_steps = 30
forecast = model_ARIMA_fit.forecast(steps=forecast_steps)

# Create a forecasted index (next 30 days)
forecast_index = pd.date_range(start=df.index[-1] + pd.Timedelta(days=0), periods=forecast_steps, freq='B')

# Step 6: Visualize the results
plt.figure(figsize=(10, 5))
plt.plot(df['Close'], label='Observed')

plt.plot(forecast_index, forecast, label='Forecast', color='red')
plt.plot(df1['Close'],label='Actual',color='Black')
plt.title('ARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
df.reset_index(inplace=True)
df.columns
forecat_res = pd.DataFrame({'Date':df1['Date'],})
# # Check the columns of df1
# print(df1.columns)

# # Assuming the correct column name is 'date' (lowercase) or something else, adjust accordingly
# # Create the DataFrame with the correct column name
# forecat_res = pd.DataFrame({'Date': df1['date']})  # Replace 'date' with the actual column name if different
 

df1['Close'].head(10)